#  Chapter 1. 신경망 복습

In [1]:
# 예제 코드를 위한 사전 작업
import numpy as np

## 벡터

In [2]:
# 벡터
x = np.array([1, 2, 3])

In [3]:
# 벡터의 형상
x.shape

(3,)

In [4]:
# 차원 수
x.ndim

1

## 행렬

In [5]:
# 행렬
W = np.array([[1, 2, 3], [4, 5, 6]])

In [6]:
# 헹렬의 형상
W.shape

(2, 3)

In [7]:
# 행렬의 차원 수
W.ndim

2

## 원소별 연산
- 각 원소가 독립적으로 대응하는 원소끼리 연산함

In [8]:
W = np.array([
    [1, 2, 3],
    [4, 5, 6]]
)
X = np.array([
    [0, 1, 2],
    [3, 4, 5]]
)

In [9]:
W + X

array([[ 1,  3,  5],
       [ 7,  9, 11]])

In [10]:
W * X

array([[ 0,  2,  6],
       [12, 20, 30]])

## 브로드캐스트
- 형상이 다른 배열끼리 연산을 할 때 '영리하게' 확장하여 연산하는 것

In [11]:
A = np.array([[1, 2], [3, 4]])
b = np.array([10, 20])

# 10이 2*2 행렬로 확장된 후에 연산을 수행
print(A * 10)

# 1차원 배열인 b가 2차원 배열로 확장된 후에 연산을 수행
print(A * b)

[[10 20]
 [30 40]]
[[10 40]
 [30 80]]


## 행렬의 내적과 곱

In [12]:
# 벡터의 내적
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

np.dot(a, b)

32

In [13]:
# 행렬의 곱
a = np.array([[1, 2], [3, 4]])
b = np.array([[5, 6], [7, 8]])

np.matmul(a, b)

array([[19, 22],
       [43, 50]])

`np.dot()` 의 인수가 모두 1차원이면 벡터의 내적을 계산함.

BUT 코드의 논리와 의도를 위해 가능하면 구분하여 코드 작성하기!

## 신경망의 추론 / 학습

- 추론
    -  문제의 답을 구하는 작업
    
- 학습
    - 최적(손실을 최소화하는)의 매개변수 값을 찾는 작업

- 계층 구현
    - 모든 계층은 `forward()` 와 `backward()` 메서드를 가진다.
    - `params` 는 가중치와 편향과 같은 매개변수를 담는 리스트이다.
    - `grads` 는 prams에 저장된 각 매개변수의 기울기를 보관하는 리스트이다.

- 손실
    - 예측이 얼마나 나쁜가를 산출한 단일 값 (스칼라)
    - 흔히 교차 엔트로피 오차(Cross Entropy Error)를 이용
    $$ L = - \sum {t_k}{log {y_k}} $$

In [14]:
class Sigmoid:
    def __init__(self):
        # sigmoid는 따로 학습하는 매개변수가 없음
        self.params = []
        self.grads = []
        self.out = None
        
    def forward(self, x):
        out = 1/ (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [15]:
class Affine:
    def __init__(self):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
        
    def forward(self, x):
        W, b = self.params
        out = np.matmul(x, W) + b
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.matmul(dout, W.T)
        dW = np.matmul(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        # ellipsis([...])
        # numpy 배열이 가리키는 메모리 위치를 고정시킨 다음 원소를 덮어씀 (Deep Copy)
        self.grad[0][...] = dW
        self.grad[1][...] = db

        return dx

## 가중치 갱신
1. 미니배치: Training Data 중에서 무작위로 다수의 데이터를 골라낸다.
2. 기울기 계산: Back-propagation으로 각 가중치 매개변수에 대한 Loss Func의 기울기를 구한다.
3. 매개변수 갱신: 기울기를 사용하여 가중치 매개변수를 갱신한다
4. 반복: 1~3단계를 반복한다

In [16]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]

## 예제 코드

In [57]:
import numpy as np
from common.layers import Affine, Sigmoid, SoftmaxWithLoss

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 가중치와 편향 초기화
        W1 = 0.01 * np.random.randn(I, H)
        b1 = np.zeros(H)
        W2 = 0.01 * np.random.randn(H, O)
        b2 = np.zeros(O)
        
        # 계층 생성
        self.layers = [Affine(W1, b1), Sigmoid(), Affine(W2, b2)]
        self.loss_layer = SoftmaxWithLoss()
        
        self.params, self.grads = [], []
        for layer in self.layers:
            print(layer)
            self.grads += layer.grads
            self.params += layer.params
            
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def forward(self, x, t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score, t)
        return loss
    
    def backward(self, dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

In [58]:
from common.optimizer import SGD
from dataset import spiral
import matplotlib.pyplot as plt

# 하이퍼파라미터 설정
max_epoch = 300
batch_size = 30
hidden_size = 10
learning_rate = 1.0

x, t = spiral.load_data()
model = TwoLayerNet(input_size=2, hidden_size=hidden_size, output_size=3)
optimizer = SGD(lr=learning_rate)

# 학습에 사용하는 변수
data_size = len(x)
max_iters = data_size // batch_size
total_loss = 0
loss_count = 0
loss_list = []

for epoch in range(max_epoch):
    # permutation 함수로 배열을 섞어줌 (idx는 data_size 크기의 배열)
    idx = np.random.permutation(data_size)
    x = x[idx]
    t = t[idx]
    
    for iters in range(max_iters):
        batch_x = x[iters*batch_size:(iters+1)*batch_size]
        batch_t = t[iters*batch_size:(iters+1)*batch_size]
            
        loss = model.forward(batch_x, batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        
        total_loss += loss
        loss_count += 1
        
        if (iters+1) % 10 == 0:
            avg_loss = total_loss / loss_count
            print("| 에폭 %d | 반복 %d / %d | 손실 %.2f" % (epoch + 1, iters + 1, max_iters, avg_loss))
            loss_list.append(avg_loss)
            total_loss, loss_count = 0, 0

| 에폭 1 | 반복 10 / 10 | 손실 1.13
| 에폭 2 | 반복 10 / 10 | 손실 1.13
| 에폭 3 | 반복 10 / 10 | 손실 1.12
| 에폭 4 | 반복 10 / 10 | 손실 1.12
| 에폭 5 | 반복 10 / 10 | 손실 1.11
| 에폭 6 | 반복 10 / 10 | 손실 1.14
| 에폭 7 | 반복 10 / 10 | 손실 1.16
| 에폭 8 | 반복 10 / 10 | 손실 1.11
| 에폭 9 | 반복 10 / 10 | 손실 1.12
| 에폭 10 | 반복 10 / 10 | 손실 1.13
| 에폭 11 | 반복 10 / 10 | 손실 1.12
| 에폭 12 | 반복 10 / 10 | 손실 1.11
| 에폭 13 | 반복 10 / 10 | 손실 1.09
| 에폭 14 | 반복 10 / 10 | 손실 1.08
| 에폭 15 | 반복 10 / 10 | 손실 1.04
| 에폭 16 | 반복 10 / 10 | 손실 1.03
| 에폭 17 | 반복 10 / 10 | 손실 0.96
| 에폭 18 | 반복 10 / 10 | 손실 0.92
| 에폭 19 | 반복 10 / 10 | 손실 0.92
| 에폭 20 | 반복 10 / 10 | 손실 0.87
| 에폭 21 | 반복 10 / 10 | 손실 0.85
| 에폭 22 | 반복 10 / 10 | 손실 0.82
| 에폭 23 | 반복 10 / 10 | 손실 0.79
| 에폭 24 | 반복 10 / 10 | 손실 0.78
| 에폭 25 | 반복 10 / 10 | 손실 0.82
| 에폭 26 | 반복 10 / 10 | 손실 0.78
| 에폭 27 | 반복 10 / 10 | 손실 0.76
| 에폭 28 | 반복 10 / 10 | 손실 0.76
| 에폭 29 | 반복 10 / 10 | 손실 0.78
| 에폭 30 | 반복 10 / 10 | 손실 0.75
| 에폭 31 | 반복 10 / 10 | 손실 0.78
| 에폭 32 | 반복 10 / 10 | 손실 0.77
| 에폭 33 | 반복 10 /

| 에폭 292 | 반복 10 / 10 | 손실 0.11
| 에폭 293 | 반복 10 / 10 | 손실 0.11
| 에폭 294 | 반복 10 / 10 | 손실 0.11
| 에폭 295 | 반복 10 / 10 | 손실 0.12
| 에폭 296 | 반복 10 / 10 | 손실 0.11
| 에폭 297 | 반복 10 / 10 | 손실 0.12
| 에폭 298 | 반복 10 / 10 | 손실 0.11
| 에폭 299 | 반복 10 / 10 | 손실 0.11
| 에폭 300 | 반복 10 / 10 | 손실 0.11


### 신경망 추론/학습에서 32비트 부동소수점으로 수행하는 이유
1. 메모리 관점에서 항상 32비트가 더 좋음
2. 데이터를 전송하는 '버스 대역폭(